# Settings

In [ ]:
%pip install mediapipe opencv-python pandas scikit-learn

In [ ]:
import mediapipe as mp
import cv2
import time
import csv
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score #accuracy score
import pickle #library to save the model

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic # Holistic model that predicts 543 landmarks

with open('fatigue.pkl', 'rb') as f:
    model = pickle.load(f) #load the model
model

# Capture Landmarks and Export to CSV

In [ ]:
import csv
import os
import numpy as np

In [ ]:
#number of landmarks
num_landmarks = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_landmarks

In [ ]:
#landmark names and their coordinates
landmarks = ['class']
for val in range(1, num_landmarks+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [ ]:
#create a csv file with the landmark names
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
class_name = "fatigue"

In [ ]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark       
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())         
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())            

            # Concate rows
            row = pose_row+face_row
            # Append class name
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
            
        except:
            pass             
                        
                        
                        
        cv2.imshow('Raw Webcam Feed', image)
        
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Model Training

## Read in Collected Data and Process

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv') #dataframe
X = df.drop('class', axis=1) #features
y = df['class'] #labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234) #split data, 30% test, 70% train


## Train ML Classification Model

In [ ]:
from sklearn.pipeline import make_pipeline #pipeline for scaling and classification
from sklearn.preprocessing import StandardScaler #scaling
from sklearn.linear_model import LogisticRegression, RidgeClassifier #classification
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier #classification 

In [ ]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()), #logistic regression
    'rc' : make_pipeline(StandardScaler(), RidgeClassifier()), #ridge classifier
    'rf' : make_pipeline(StandardScaler(), RandomForestClassifier()), #random forest
    'gb' : make_pipeline(StandardScaler(), GradientBoostingClassifier()) #gradient boosting
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, y_train) #fit the model
    fit_models[algo] = model #add the model to the dictionary

In [ ]:
fit_models['rf'].predict(X_test.values) #predict the test data using logistic regression

## Evaluate and Serialize Model

In [ ]:
from sklearn.metrics import accuracy_score #accuracy score
import pickle #library to save the model

In [ ]:
for algo, model in fit_models.items(): #for each model in the dictionary
    yhat = model.predict(X_test) #predict the test data
    print(algo, accuracy_score(y_test, yhat)) #print the accuracy score
    with open('fatigue.pkl', 'wb') as f:
        pickle.dump(fit_models['rf'], f) #save the model as a pickle file

# Make Detection with My Model

In [ ]:
with open('fatigue.pkl', 'rb') as f:
    model = pickle.load(f) #load the model

In [ ]:
model

In [ ]:
cap = cv2.VideoCapture(0)

# Variables for eyes detection
eye_closed = False
eye_closed_start_time = 0
eye_closed_duration = 0
blink_counter = 0

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        

        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark       
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())         
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())            

            # Concate rows
            row = pose_row+face_row
            
            # # Append class name
            # row.insert(0, class_name)
            
            # # Export to CSV
            # with open('coords.csv', mode='a', newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row)
            
            # Make Detections
            x = pd.DataFrame([row])
            model_class = model.predict(x)[0] #class of the model
            model_prob = model.predict_proba(x)[0] #probability of each class
            #print(model_class, model_prob)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, model_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(model_prob[np.argmax(model_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            #Detect eye landmarks
            if results.face_landmarks is not None:
                left_eye_landmarks = [results.face_landmarks.landmark[33], results.face_landmarks.landmark[133], results.face_landmarks.landmark[159], results.face_landmarks.landmark[145], results.face_landmarks.landmark[153], results.face_landmarks.landmark[157]]
                right_eye_landmarks = [results.face_landmarks.landmark[362], results.face_landmarks.landmark[263], results.face_landmarks.landmark[386], results.face_landmarks.landmark[374], results.face_landmarks.landmark[380], results.face_landmarks.landmark[382]]
                # Calculate vertical distance between top and bottom of left eye
                left_eye_top = left_eye_landmarks[0].y
                left_eye_bottom = left_eye_landmarks[3].y
                left_eye_vertical_distance = left_eye_bottom - left_eye_top
                right_eye_top = right_eye_landmarks[0].y
                right_eye_bottom = right_eye_landmarks[3].y
                right_eye_vertical_distance = right_eye_bottom - right_eye_top
                # Calculate horizontal distance between left and right of left eye
                left_eye_left = left_eye_landmarks[4].x
                left_eye_right = left_eye_landmarks[5].x
                left_eye_horizontal_distance = left_eye_right - left_eye_left
                right_eye_left = right_eye_landmarks[4].x
                right_eye_right = right_eye_landmarks[5].x
                right_eye_horizontal_distance = right_eye_right - right_eye_left
                # Calculate eye aspect ratio
                left_eye_aspect_ratio = left_eye_vertical_distance / left_eye_horizontal_distance
                right_eye_aspect_ratio = right_eye_vertical_distance / right_eye_horizontal_distance
                # Calculate average eye aspect ratio
                average_eye_aspect_ratio = (left_eye_aspect_ratio + right_eye_aspect_ratio) / 2
                # Detect if eyes are closed
                if average_eye_aspect_ratio < 0.2:
                    if not eye_closed: #if eyes were open before, start timer
                        eye_closed_start_time = time.time() 
                    eye_closed = True
                    cv2.putText(image, 'Eyes Closed', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                else:
                    if eye_closed:
                        eye_closed_duration = time.time() - eye_closed_start_time
                        print('Eyes were closed for ' + str(round(eye_closed_duration, 2)) + ' seconds')
                    cv2.putText(image, 'Eyes Open', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    eye_closed = False           
                   
                
                
            
        except:
            pass             
                        
        
                        
        cv2.imshow('Raw Webcam Feed', image)
        
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()